# geom #2
Fully Restrained Beam

In [ ]:
import deepxde as dde
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
L = 1;   E = 200e9;   I = 1e-6;   q = 10000.0;   c = q / (E * I)
geom2 = dde.geometry.Interval(0, L)

In [ ]:
def ode2(x, y):
    dy_xx = dde.grad.hessian(y, x)
    dy_xxxx = dde.grad.hessian(dy_xx, x)
    return dy_xxxx + c

def solution(x):
    return (-q / (24 * E * I)) * (x**4 - 2 * L * x**3 + L**2 * x**2)

In [ ]:
def boundary_l(x, on_boundary):
    return on_boundary and dde.utils.isclose(x[0], 0)

def boundary_r(x, on_boundary):
    return on_boundary and dde.utils.isclose(x[0], L)

bcD_l = dde.icbc.DirichletBC(geom2, lambda x: 0, boundary_l)
bcN_l = dde.icbc.NeumannBC(geom2, lambda x: 0, boundary_l)

bcD_r = dde.icbc.DirichletBC(geom2, lambda x: 0, boundary_r)
bcN_r = dde.icbc.NeumannBC(geom2, lambda x: 0, boundary_r)

bcs2 = [bcD_l, bcN_l, bcD_r, bcN_r]

In [ ]:
data2 = dde.data.PDE(geom2, ode2, bcs2, num_domain=100, num_boundary=50, solution=solution)

In [ ]:
lw = [1, 1, 1, 1, 1]
net2 = dde.nn.FNN([1, 50, 50, 50, 1], "swish", "Glorot uniform")
model2 = dde.Model(data2, net2)
model2.compile("adam", lr=1e-4, metrics=['l2 relative error'], loss_weights=lw)
model2.train(iterations=10000, display_every=1000)
model2.compile("L-BFGS", loss_weights=lw)
model2.train()

In [ ]:
X2 = np.linspace(0, L, 200)[:, None]
y_pred2 = model2.predict(X2)
w_analytical2 = solution(X2)


plt.figure(figsize=(10, 6))
plt.plot(X2, y_pred2, label="PINN Prediction", color="red", linestyle="--")
plt.plot(X2, w_analytical2, label="Exact Solution", color="blue")
plt.xlabel("x")
plt.ylabel("w(x)")
plt.title("PINN vs Analytical: Deflection of Fully Restrained Beam Under Uniform Load")
plt.legend()
plt.grid()
plt.show()